# 네이버 뉴스 크롤링 코드 (기능)
## 요구 사항
### - 지역명 입력 받아 글자 잘라서 자동 검색 되도록
####    - 예: 경상북도 안동시 -> 안동 / 경상남도 창원시 마산합포구 -> 마산합포
### - 구현 파트에 크롤링 결과를 어떤 형태로 넘겨줘야 하는지 상의 필요

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup     
import pandas as pd   
import openpyxl
import urllib.request
import urllib
import time
import sys
import re
import math
import os
import random

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

# 사용자로부터 입력 받기
query_txt = input('키워드 : ')
cnt=int(input('데이터 건 수: ') )
page_cnt = math.ceil(cnt / 10)

f_dir = input('저장 경로 (예:c:\\py_temp\\): ')
if f_dir =='' :
        f_dir = "c:\\py_temp\\" + query_txt + '\\'
#fc_name=input('csv 저장 경로와 파일명(예:c:\\py_temp\\naver.csv): ')
#fx_name=input('xls 저장 경로와 파일명(예:c:\\py_temp\\naver.xls): ')
fc_name = f_dir + 'Naver news' + query_txt + '.csv'
fx_name = f_dir + 'Naver news' + query_txt + '.xls'
print('=' *100)

# 네이버 접속 -> 뉴스 페이지로 이동
url = 'https://www.naver.com'
driver.get(url)
driver.maximize_window()
time.sleep(2)
s_time = time.time( )
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)
driver.find_element(By.LINK_TEXT,'뉴스').click( )
time.sleep(1)
driver.find_element(By.LINK_TEXT, '옵션').click()
time.sleep(1)
driver.find_element(By.LINK_TEXT, '보도자료').click()

# 저장할 리스트
title2 = []        # 제목
img_src2=[]         # 이미지
url_all_list=[]    # URL

img_dir = f_dir + ' photo' + '\\' # 이미지는 csv, xls 파일이 저장된 폴더 내의 photo라는 폴더 내에 저장
os.makedirs(img_dir)
os.chdir(img_dir)
no = 1

# 크롤링
for a in range(1, page_cnt + 1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    news_list = soup.find('ul','list_news').find_all('li')
        
    for i in news_list:
        try:
            lump = i.find('div','news_area').find_all('a') 
        except:
            continue
            
        img_src = i.find_all('img')

        url_all_list.append(lump[5]['href'])
        print('주소:', lump[5]['href'])
        
        title = lump[4].get_text()              # 게시물 제목
        if title == '네이버뉴스':
            title = lump[5].get_text()
        title2.append(title)                       # 게시물 제목 리스트에 추가
        print('제목:',title)
        
        img_src1 = img_src[1]['src']
        if 'http' in img_src1 : # 찾는 이미지는 http로 시작하는 이미지인데 이렇게 시작하지 않는 이미지(예를 들어 버튼..)은 거르기 위해 이 if문 사용
            img_src2.append(img_src1)
            
        print('이미지:', img_src1)
        print("\n")
        
        no += 1
        
        if no > cnt :
            break

file_no = 1
for i in range(0,len(img_src2)) :
        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except :
                continue        
        time.sleep(0.5)      
        file_no += 1  
        if file_no > cnt :
            break
            
# 출력 결과를 표(데이터 프레임) 형태로 만들기
naver_news = pd.DataFrame()
naver_news['제목'] = title2
#naver_news['사진'] = img_src2
naver_news['주소'] = url_all_list

# csv 형태로 저장하기
naver_news.to_csv(fc_name,encoding="utf-8-sig",index=False)
print(" csv 파일 저장 경로: %s" %fc_name) 

# 엑셀 형태로 저장하기
naver_news.to_excel(fx_name , index=False , encoding="UTF-8" , engine='openpyxl')
print(" xls 파일 저장 경로: %s" %fx_name) 